# Boxcars Gem Notebook
Below you will find some sample boxcar gem code fragments.

## Env Setup
Before we get started, install the gems we need here:
```bash
gem install boxcars dotenv
```
and then create / edit .env to have SERPAPI_API_KEY and OPENAI_ACCESS_TOKEN

In [1]:
# setup for code below
require "dotenv/load"
require "boxcars"

true

## Examples
Here are several examples of using the boxcar gems. First we will start with individual Boxcars, and then we will move on to using Trains.

### Search using the Google Serp API (set SERPAPI_API_KEY in .env)

In [2]:
# showcase Google Serp search
s = Boxcars::GoogleSearch.new
s.run "what temperature is it in Phoenix?"

Question: what temperature is it in Phoenix?
Answer: High 66F


"High 66F"

### Calculator that uses ruby for hard math

In [3]:
c = Boxcars::Calculator.new
c.run "what is pi to the third power times epsilon?"

> Entering Calculator#run
what is pi to the third power times epsilon?
have error, trying again: Error: expecting your response to begin with '```ruby'. Try answering the question again.
RubyREPL: epsilon = 2.71828 # replace with your value of epsilon
puts Math::PI ** 3 * epsilon
Answer: 84.28374177452538

{"status":"ok","answer":"84.28374177452538","explanation":"Answer: 84.28374177452538","code":"epsilon = 2.71828 # replace with your value of epsilon\nputs Math::PI ** 3 * epsilon"}
< Exiting Calculator#run


"84.28374177452538"

### Active Record Example using SQL code generation

In [4]:
# first stub out an in memory sqlite3 and a simple helpdesk example
require 'active_record'
require './helpdesk_sample'

-- create_table("users", {:force=>:cascade})
   -> 0.0083s
-- create_table("comments", {:force=>:cascade})
   -> 0.0005s
-- create_table("tickets", {:force=>:cascade})
   -> 0.0003s


true

In [5]:
# get the answer
boxcar = Boxcars::SQL.new
boxcar.run "How many tickets are there?"

> Entering Database#run
How many tickets are there?
SELECT COUNT(*) FROM tickets;
{"status":"ok","answer":3,"explanation":"Answer: 3","code":"SELECT COUNT(*) FROM tickets;"}
< Exiting Database#run


3

In [7]:
boxcar.run "How many comments do we have on tickets with a status of 1?"

> Entering Database#run
How many comments do we have on tickets with a status of 1?
SELECT COUNT(*) FROM comments JOIN tickets ON comments.ticket_id = tickets.id WHERE tickets.status = 1;
{"status":"ok","answer":0,"explanation":"Answer: 0","code":"SELECT COUNT(*) FROM comments JOIN tickets ON comments.ticket_id = tickets.id WHERE tickets.status = 1;"}
< Exiting Database#run


0

### Active Record Example using ActiveRecord code generation

In [8]:
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
helpdesk.run "how many comments do we have on open tickets?"

> Entering helpdesk#run
how many comments do we have on open tickets?
Comment.joins(:ticket).where(tickets: {status: "open"}).count
{"status":"ok","answer":4,"explanation":"Answer: 4","code":"Comment.joins(:ticket).where(tickets: {status: \"open\"}).count"}
< Exiting helpdesk#run


4

#### ActiveRecord is Read Only by default

In [9]:
helpdesk.run "Move all of Sally's open tickets to John"

> Entering helpdesk#run
Move all of Sally's open tickets to John
computing change count with: Ticket.where(user_id: User.find_by(name: 'Sally').id, status: 'open').update_all(user_id: User.find_by(name: 'John').id)
helpdesk(Pending Changes): 2
Can not run code that makes 2 changes in read-only mode
Permission to run code that makes changes denied
< Exiting helpdesk#run


"Error: Permission to run code that makes changes denied"

#### But you can make changes if you want!

In [10]:
rw_helpdesk = Boxcars::ActiveRecord.new(read_only: false, name: 'Helpdesk', models: [Ticket, User, Comment])
rw_helpdesk.run "Move all of Sally's open tickets to John"

> Entering Helpdesk#run
Move all of Sally's open tickets to John
computing change count with: Ticket.where(user_id: User.find_by(name: 'Sally').id, status: 'open').update_all(user_id: User.find_by(name: 'John').id)
Helpdesk(Pending Changes): 2
Ticket.where(user_id: User.find_by(name: 'John').id).where(status: 'open')
{"status":"ok","answer":[{"id":1,"title":"First ticket","user_id":1,"status":"open","body":"This is the first ticket","created_at":"2023-03-20T22:58:53.026Z","updated_at":"2023-03-20T22:58:53.026Z"}],"explanation":"Answer: [{\"id\":1,\"title\":\"First ticket\",\"user_id\":1,\"status\":\"open\",\"body\":\"This is the first ticket\",\"created_at\":\"2023-03-20T22:58:53.026Z\",\"updated_at\":\"2023-03-20T22:58:53.026Z\"}]","code":"Ticket.where(user_id: User.find_by(name: 'John').id).where(status: 'open')"}
< Exiting Helpdesk#run


[{"id"=>1, "title"=>"First ticket", "user_id"=>1, "status"=>"open", "body"=>"This is the first ticket", "created_at"=>"2023-03-20T22:58:53.026Z", "updated_at"=>"2023-03-20T22:58:53.026Z"}]

#### Or you can use a callback for approval if changes are needed

In [11]:
Ticket.update_all(user_id: User.find_by(name: 'John').id)
my_approver = Proc.new do |change_count, code|
    puts ">>> Approving #{change_count} changes <<<"
    true
end
ap_helpdesk = Boxcars::ActiveRecord.new(approval_callback: my_approver, name: 'Helpdesk', models: [Ticket, User, Comment])
ap_helpdesk.run "Move all of John's open tickets to Sally"

> Entering Helpdesk#run
Move all of John's open tickets to Sally
computing change count with: Ticket.where(user_id: john.id, status: 'open').update_all(user_id: sally.id)
Error while running code: undefined local variable or method `john' for #<Boxcars::Acti ...
have error, trying again: ARChanges Error: undefined local variable or method `john - please fix "ARChanges:" to not have this error
computing change count with: Ticket.where(user_id: User.find_by(name: 'John').id, status: 'open').update_all(user_id: User.find_by(name: 'Sally').id)
Helpdesk(Pending Changes): 3
>>> Approving 3 changes <<<
Ticket.where(user_id: User.find_by(name: 'John').id, status: 'open').update_all(user_id: User.find_by(name: 'Sally').id)
{"status":"ok","answer":3,"explanation":"Answer: 3","code":"Ticket.where(user_id: User.find_by(name: 'John').id, status: 'open').update_all(user_id: User.find_by(name: 'Sally').id)"}
< Exiting Helpdesk#run


3

## Putting it all together - Trains
Trains uses a series of Boxcars to compute an answer by composing answers from the union of queries to individual boxcars as needed.
### a one car train
This is similar to just running Boxcar.run

In [12]:
c = Boxcars::Calculator.new
Boxcars.configuration.log_prompts = true
train = Boxcars.train.new(boxcars: [c])
puts train.run "what is pi squared?"

> Entering Zero Shot#run
what is pi squared?
>>>>>> Role: system <<<<<<
Answer the following questions as best you can. You have access to the following actions:
Calculator: useful for when you need to answer questions about math
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one from this list: [Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation sequence can repeat N times)
Thought: I know the final answer
Final Answer: the final answer to the original input question
Next Actions: Up to 3 logical suggested next questions for the user to ask after getting this answer.
Remember to start a line with "Final Answer:" to give me the final answer.
Begin!
>>>>>> Role: user <<<<<<
Question: what is pi squared?
>>>>>> Role: assistant <<<<<<
Thought: 
Thought: I know that pi is a mathematical constan

### a two car train
Here we have a HelpDesk ActiveRecord database and a calculator at our disposal to get an answer

In [13]:
c = Boxcars::Calculator.new
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
train = Boxcars.train.new(boxcars: [c, helpdesk])
train.run "the number of open helpdesk tickets that John commented on times 2 pi?"

> Entering Zero Shot#run
the number of open helpdesk tickets that John commented on times 2 pi?
>>>>>> Role: system <<<<<<
Answer the following questions as best you can. You have access to the following actions:
Calculator: useful for when you need to answer questions about math
helpdesk: useful for when you need to query a database for an application named .
Use the following format:
Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one from this list: [Calculator, helpdesk]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation sequence can repeat N times)
Thought: I know the final answer
Final Answer: the final answer to the original input question
Next Actions: Up to 3 logical suggested next questions for the user to ask after getting this answer.
Remember to start a line with "Final Answer:" to give me the final answer.
Begin!
>

The final answer is 12.56.

Final Answer: 12.56

Next Actions:
1. What are the details of the two open tickets that John commented on?
2. How many open tickets are there in total?
3. What is the average response time for open tickets that John commented on?
< Exiting Zero Shot#run


"12.56\n\nNext Actions:\n1. What are the details of the two open tickets that John commented on?\n2. How many open tickets are there in total?\n3. What is the average response time for open tickets that John commented on?"

#### a three car train

In [16]:
c = Boxcars::Calculator.new
search = Boxcars::GoogleSearch.new
helpdesk = Boxcars::ActiveRecord.new(name: 'helpdesk', models: [Ticket, User, Comment])
train = Boxcars.train.new(boxcars: [c, helpdesk, search])
train.run "the number of open helpdesk tickets that John commented on times 2 pi plus the temp in Phoenix"

> Entering Zero Shot#run
the number of open helpdesk tickets that John commented on times 2 pi plus the temp in Phoenix
> Entering helpdesk#run
Search for John's comments in the helpdesk tickets
Comment.where(user_id: User.find_by(name: 'John').id)
Answer: #<ActiveRecord::Relation [#<Comment id: 1, content: "This is a comment", user_id: 1, ticket_id: 1, created_at: "2023-02-23 23:40:50.536049000 +0000", updated_at: "2023-02-23 23:40:50.536049000 +0000">, #<Comment id: 2, content: "This is johns second comment", user_id: 1, ticket_id: 1, created_at: "2023-02-23 23:40:50.536994000 +0000", updated_at: "2023-02-23 23:40:50.536994000 +0000">]>
< Exiting helpdesk#run
Observation: Answer: #<ActiveRecord::Relation [#<Comment id: 1, content: "This is a comment", user_id: 1, ticket_id: 1, created_at: "2023-02-23 23:40:50.536049000 +0000", updated_at: "2023-02-23 23:40:50.536049000 +0000">, #<Comment id: 2, content: "This is johns second comment", user_id: 1, ticket_id: 1, created_at: "2023-02-23

"68.2831853071796 open helpdesk tickets that John commented on times 2 pi plus the temp in Phoenix"

#### Add as many as you want.